# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [11]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [17]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [16]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [5]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [18]:
def foldl2(f, x0, lst):
    return foldr(lambda x, g: lambda y: g(f(y, x)), lambda x: x, lst)(x0)

foldl2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [19]:
def foldr2(f, x0, lst):
    return foldl(lambda g, x: lambda y: g(f(x, y)), lambda x: x, lst)(x0)

foldr2(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [14]:
from typing import List

def check_inv(a, b):
    #защита
    if (len(b) > len(a)):
        return False

    #делаем список списков с количеством встреч каждой буквы в b
    l1: List[List] = []
    for i in range (len(b)):
        if not [b[i], b.count(b[i])] in l1:
            l1.append([b[i], b.count(b[i])])

    l2: List[List] = []
    #итерируемся по куску длиной строки b в строке a
    for i in range (len(a) - len(b) + 1):
        subStr = a[i:i+len(b)]
        #список с кол-вом встреч букв в куске
        for j in range (len(b)):
            if not [subStr[j], subStr.count(subStr[j])] in l2:
                l2.append([subStr[j], subStr.count(subStr[j])])
        #сравниваем со списком из b
        if sorted(l1, key=lambda x: x[0]) == sorted(l2, key=lambda x: x[0]):
            return True
        l2 = []
    return False

a = 'абвгдеёж'
b = 'едг'
check_inv(a, b)

#сложность тут, по-моему, получается O((a-b)*b)

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [22]:
class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value
        
    def __iter__(self):
        if (self.left == None and self.right == None):
            yield self.value
        else:
            if (self.left):
                yield from self.left
            if (self.right):
                yield from self.right
    
    def __str__(self):
    if (self.left == None and self.right == None):
        return 'Tree ({})'.format(self.value)
    leftStr = self.left.__str__() or ''
    rightStr = self.right.__str__() or ''
    return leftStr + rightStr
    
    def __repr__(self):
        return 'Tree ({})'.format(self.value)
    
tree = Tree(0, Tree(1, Tree(3), Tree(4)),                             
               Tree(2))

list(tree) == [3, 4, 2]

True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [38]:
def calc(expr):
    #ищем скобки
    openingBracket = expr.find('(')
    if openingBracket != -1:        #если нашли скобку
        closingBracket = expr.rfind(')')      #ищем справа закрывающую скобку
        calculatedExpr = calc(expr[openingBracket + 1:closingBracket])       #вызываем функцию подсчета того, что внутри скобок
        expr = expr.replace(expr[openingBracket:closingBracket + 1], str(calculatedExpr)) #подставили посчитанное и убрали скобки
        return calc(expr)

    #ищем плюс
    elif expr.find('+') != -1:        #если нашли плюс
        plusIndex = expr.find('+')    #запоминаем место плюса
        leftExpr = calc(expr[0:plusIndex])     #считаем всё, что слева от плюса
        rightExpr = calc(expr[plusIndex + 1:len(expr)]) #считаем всё, что справа от плюса
        return leftExpr + rightExpr            #возвращаем значение, сложив л.часть и пр.части


    #ищем минус
    elif expr.find('-') != -1:          #если нашли минус
        minusIndex = expr.rfind('-')    #запоминаем место минуса
        leftExpr = calc(expr[0:minusIndex])              #считаем всё, что слева от минуса
        rightExpr = calc(expr[minusIndex + 1:len(expr)]) #считаем всё, что справа от минуса
        return leftExpr - rightExpr                      #возвращаем значение, вычитая из л.ч.  пр.ч.

    #ищем умножение
    elif expr.find('*') != -1:          #если нашли умножение
        starIndex = expr.find('*')      #запоминаем индекс знака умножения
        leftExpr = calc(expr[0:starIndex])              #считаем всё, что слева от умножения
        rightExpr = calc(expr[starIndex + 1:len(expr)]) #считаем всё, что справа от умножения
        return leftExpr * rightExpr                     #возвращаем значение, умножив л.ч. на пр.ч.

    #ищем деление
    elif expr.find('/') != -1:               #если нашли деление
        slashIndex = expr.rfind('/')         #запоминаем индекс знака деления
        leftExpr = calc(expr[0:slashIndex])  #считаем всё, что слева от деления
        rightExpr = calc(expr[slashIndex + 1:len(expr)]) #считаем всё, что справа от деления
        return int(leftExpr / rightExpr)                 #возвращаем значение, поделив л.ч. на пр.ч. (и округляем вниз)

    #ничего не нашли - возвращаем значение
    else:
        return int(expr)



calc('2 * (15 - 3 * 4) - 2') == 4

print(calc('3 + (4 * (15 / 5)) * 2'))

print(calc('(75 * (80 / (40 / 2))) * 3'))

print(calc('111 / 3'))

27
900
37
